In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.8MB/s 
     |████████████████████████████████| 8.2MB 6.4MB/s 


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import pymorphy2 
import datetime
import nltk
import pickle
nltk.download('punkt')

from sklearn.metrics import accuracy_score, f1_score, recall_score
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import files

In [5]:
files.upload()

Saving Переписка_По_Проблемам.xlsx to Переписка_По_Проблемам.xlsx
Saving Проблемы с фото в ответе.xlsx to Проблемы с фото в ответе.xlsx
Saving Список_Проблем.xlsx to Список_Проблем.xlsx


In [ ]:
troubles = pd.read_excel("Список_Проблем.xlsx")
photo_in_answer = pd.read_excel("Проблемы с фото в ответе.xlsx")
msg = pd.read_excel("Переписка_По_Проблемам.xlsx")

In [18]:
df = pd.DataFrame()

TypeError: ignored

In [ ]:
df['words'] = ''
df['count'] = 0

In [68]:
troubles['responsibleperson_msg'] = ' '
for i in range(0, len(msg)):

  if (msg.person__polymorphic_ctype__model[i] == 'responsibleperson') :

    troubles['responsibleperson_msg'][troubles['entity_id'] ==  msg['entity__id'][i]] += ' | ' + str(msg['description'][i])

In [77]:
data = troubles

In [79]:
morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
for i in range(len(data['responsibleperson_msg'])):
    data['responsibleperson_msg'][i] = re.sub(patterns,'',data['responsibleperson_msg'][i])
    data['responsibleperson_msg'][i] = data['responsibleperson_msg'][i].split()
    for j in range(len(data['responsibleperson_msg'][i])):
      data['responsibleperson_msg'][i][j] = data['responsibleperson_msg'][i][j].strip()
      data['responsibleperson_msg'][i][j] = morph.normal_forms(data['responsibleperson_msg'][i][j])
      

In [88]:
data['target'] = 1
data['target'][pd.isna(data['rework_at'])] = 0 

In [91]:
arr = []

In [101]:
for i in range(len(data)):
  if (i % 100 == 0):
    print(i)
  if (data['target'][i]):
    for elem in data['responsibleperson_msg'][i]:

      if type(elem) == type([]):
        
        f = -1
        for j in range(0, len(arr)):

          if (arr[j][0] == elem[0]):

            f = j
            break

        if f != -1:

          arr[j][1] += 1

        else:

          arr.append([elem[0], 1])

          

      else:

        f = -1
        for j in range(0, len(arr)):

          if (arr[j][0] == elem):

            f = j

        if f != -1:

          arr[j][1] += 1

          break

        else:

          arr.append([elem, 1])




0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300


In [103]:
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

In [104]:
def my_cmp(x, y):
    return x[1] - y[1]

In [105]:
new_list = sorted(arr, key=cmp_to_key(my_cmp))

In [108]:
words = pd.DataFrame(new_list[-90:])

In [110]:
words.columns = ['Слово','Количество вхождений']

In [138]:
for word in words['Слово']:
  print(word)

сообщение
день
связь
муниципальный
жкх
благоустройство
освещение
объект
повторно
настоящий
улица
направить
провести
развитие
разместить
место
срок
проведение
установить
округ
расположить
следующий
который
транспорт
дорога
управлять
далее
участок
время
собственник
жилой
строительство
рассмотрение
дорожный
помещение
результат
общий
комитет
город
соответствие
управление
вышеуказанный
белгород
департамент
хозяйство
состояние
область
информация
адрес
многоквартирный
выполнить
указанный
организация
ремонт
городской
администрация
район
рассмотреть
белгородский
быть
сообщать
территория
ваш
ул
вопрос
№
работа
дом
год
уважаемый
обращение


Удалим часть стоп слов////


In [140]:
words = words[words['Слово'] != 'ул']

In [141]:
words.to_excel("Most_frequent_words.xlsx")

In [142]:
files.download("Most_frequent_words.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>